In [12]:
from autogen import ConversableAgent
from dotenv import load_dotenv
import os

In [ ]:
load_dotenv()

agent = ConversableAgent("chatbot",
                          llm_config={"model": "gpt-4.1-mini", "api_key": os.environ.get("OPENAI_API_KEY")},
                        human_input_mode="NEVER")

reply = agent.generate_reply(messages=[{"content": "Tell me a joke", "role": "user"}])

print(reply)

Sure! Here's a joke for you:

Why don’t scientists trust atoms?

Because they make up everything!
